In [2]:
"""
author: Hecate
"""
import pandas as pd
tagging = pd.read_excel('taggings.xlsx')
tag = pd.read_excel('tags.xlsx')
eign = pd.read_excel('Eigen56.xlsx')
article = pd.read_excel('Articles56.xlsx')
daily = pd.read_excel('Dailies56.xlsx')

def merge(dfo):
    df = pd.merge(dfo,tagging,left_on='id',right_on='taggable_id',how='inner')
    df1 = pd.merge(df,tag,left_on='tag_id',right_on='id',how='inner') \
    .drop('id',1).rename(columns={'id_x':'id'})
    df1 = df1[['id','title','content','name']]
    df2 = df1.groupby(['id']).agg({'name':lambda x: set(x), \
                                'title':'first','content':'first'}).reset_index()
    return df2

merge(article).to_excel('article_tag_hecate.xlsx')
merge(daily).to_excel('daily_tag_hecate.xlsx')

In [108]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jul  9 13:14:52 2019

@author: rain
"""

# 导入tags and tagging 数据
import pandas as pd
tagging = pd.read_excel('taggings.xlsx', index_col=None)
tags = pd.read_excel('tags.xlsx', index_col=None)
tags = tags[['id', 'name']]


#  再进行long to wide 的转变 
def long_to_wide(df):
    title = df['title'].iloc[0]
    content = df['content'].iloc[0]
    tags = [list(df.name)]
    new_df = pd.DataFrame({'title': title, 'content': content, 'tags': tags})
    return(new_df)




def merge(typ, data):
    '''
    typ 是tagging 这个dataframe里面'taggable_type'的类型，可以是['Article', 'Video', 'Topic', 'Daily', 'Document', 'Event'] 等
    data 是导入的因为数据，包括：daily 和 articles
    return - 合并好的数据集
    '''
    daily_tagging = tagging[tagging['taggable_type'] == typ][['tag_id', 'taggable_id']]
    main_tags = pd.merge(left = daily_tagging, right = tags, right_on = 'id', left_on = 'tag_id', how = 'inner')
    
    daily_merge = pd.merge(left = data, right = main_tags, left_on = 'id', right_on = 'taggable_id', how ='left')
    daily_merge = daily_merge[['id_x', 'title', 'content', 'name']]
    daily_merge['name'] = daily_merge['name'].fillna(' ') 
    
    new = long_to_wide(daily_merge.iloc[:4,:])   
    for ele in list(daily_merge['id_x'].unique()):
        df = daily_merge[daily_merge['id_x'] == ele]
        new_df = long_to_wide(df)
        new = pd.concat([new, new_df])
        
    new = new.drop_duplicates(subset = 'title')
    return (new)
dfa = merge('Article',article)